In [1]:
!pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 27.8 MB/s eta 0:00:00


In [2]:
import pandas as pd
import xml.etree.ElementTree as ET
from Bio import SeqIO
from Bio import Entrez
Entrez.email = "A.N.Other@example.com"

In [3]:
from google.colab import files
uploaded = files.upload()

Saving disease.tsv to disease.tsv


In [4]:
dis = pd.read_table('disease.tsv')
dis.head(4)

,geneId,geneSymbol,DSI,DPI,diseaseId,diseaseName,diseaseType,diseaseClass,diseaseSemanticType,score,EI,YearInitial,YearFinal,NofPmids,NofSnps,source
0,1,A1BG,0.700,0.538,C0019209,Hepatomegaly,phenotype,C23;C06,Finding,0.30,1.000,2017.0,2017.0,1,0,CTD_human
1,1,A1BG,0.700,0.538,C0036341,Schizophrenia,disease,F03,Mental or Behavioral Dysfunction,0.30,1.000,2015.0,2015.0,1,0,CTD_human
2,2,A2M,0.529,0.769,C0002395,Alzheimer's Disease,disease,C10;F03,Disease or Syndrome,0.50,0.769,1998.0,2018.0,3,0,CTD_human
3,2,A2M,0.529,0.769,C0007102,Malignant tumor of colon,disease,C06;C04,Neoplastic Process,0.31,1.000,2004.0,2019.0,1,0,CTD_human


In [5]:
schis_db = dis.loc[dis['diseaseName'] == "Alzheimer's Disease"]
schis_db.head()

,geneId,geneSymbol,DSI,DPI,diseaseId,diseaseName,diseaseType,diseaseClass,diseaseSemanticType,score,EI,YearInitial,YearFinal,NofPmids,NofSnps,source
2,2,A2M,0.529,0.769,C0002395,Alzheimer's Disease,disease,C10;F03,Disease or Syndrome,0.50,0.769,1998.0,2018.0,3,0,CTD_human
365,43,ACHE,0.445,0.885,C0002395,Alzheimer's Disease,disease,C10;F03,Disease or Syndrome,0.40,0.985,1991.0,2020.0,2,0,CTD_human
818,102,ADAM10,0.489,0.846,C0002395,Alzheimer's Disease,disease,C10;F03,Disease or Syndrome,0.70,0.986,2000.0,2019.0,1,1,CTD_human
2760,267,AMFR,0.599,0.692,C0002395,Alzheimer's Disease,disease,C10;F03,Disease or Syndrome,0.31,1.000,2012.0,2019.0,1,0,CTD_human
2790,274,BIN1,0.550,0.692,C0002395,Alzheimer's Disease,disease,C10;F03,Disease or Syndrome,0.50,0.980,2011.0,2020.0,1,6,CTD_human


In [6]:
characteristic = schis_db.iloc[[2],[1,2,3,5]]
characteristic

,geneSymbol,DSI,DPI,diseaseName
818,ADAM10,0.489,0.846,Alzheimer's Disease


In [ ]:
handle = Entrez.efetch(db="gene", id="102", rettype = 'xml')
gene_locus = ET.parse(handle).getroot().find("Entrezgene/Entrezgene_locus")
region = gene_locus.find("Gene-commentary/Gene-commentary_seqs/Seq-loc/Seq-loc_int/Seq-interval")
start = int(region.find("Seq-interval_from").text) + 1
end = int(region.find("Seq-interval_to").text) + 1
gi_id = region.find("Seq-interval_id/Seq-id/Seq-id_gi").text
strand = region.find("Seq-interval_strand/Na-strand").get("value")
print(start, end, gi_id, strand)

58588809 58749707 568815583 minus


In [7]:
handle = Entrez.efetch(db="nucleotide",
                       id="NC_000015.10",
                       rettype = 'fasta',
                       retmode = 'text')

chromosome = SeqIO.read(handle, 'fasta')

In [8]:
chromosome.description

'NC_000015.10 Homo sapiens chromosome 15, GRCh38.p14 Primary Assembly'

In [9]:
geneADAM10 = chromosome.seq[58588809:58749707]
print(geneADAM10)
len(geneADAM10)

...


160898

In [12]:
handle = Entrez.efetch(db="gene", id="43", rettype = 'xml')
gene_locus = ET.parse(handle).getroot().find("Entrezgene/Entrezgene_locus")
region = gene_locus.find("Gene-commentary/Gene-commentary_seqs/Seq-loc/Seq-loc_int/Seq-interval")
start = int(region.find("Seq-interval_from").text) + 1
end = int(region.find("Seq-interval_to").text) + 1
gi_id = region.find("Seq-interval_id/Seq-id/Seq-id_gi").text
strand = region.find("Seq-interval_strand/Na-strand").get("value")
print(start, end, gi_id, strand)

100889994 100896994 568815591 minus


In [13]:
handle = Entrez.efetch(db="nucleotide",
                       id="NC_000007.14",
                       rettype = 'fasta',
                       retmode = 'text')

chromosome = SeqIO.read(handle, 'fasta')

In [18]:
chromosome.description

'NC_000007.14 Homo sapiens chromosome 7, GRCh38.p14 Primary Assembly'

In [14]:
geneACHE = chromosome.seq[100889994:100896994]
print(geneACHE)
len(geneACHE)

GGCTGTAACAGTTTATTGGCAGCCCAGAGGGGCGAAGGCACCGCGGGGGAGGGAGCTCAGCCTGAGACATGCAGAGGACCGGGAGCCCCGGGGGACGTCGGGGTGGGGTGGGGATGGGCAGAGTCTGGGGCTCGTCTGTGTTATAGCCCAGCCCTGAAATAAATAGTATATACAGCTAGGGGGCCGGGCGGAGCGGAGGACATGGGGGTCCCGCCGGGGTCACAGGTCTGAGCAGCGATCCTGCTTGCTGTAGTGGTCGAACTGGTTCTTCCAGTGCACCATGTAGGAGCTCCAGCGGTGGAACTCGGCCTTCCACTGGCGCTCCGCCTCGTCGAGCGTGTCTGCGGCCAGGGCGCCCAGCGAGGCGGGAGGGGAGGACGGCACGAGGAAGGTCGGGGATCGGACATTTGGGCGGGTTGAAGGGGGGGTATGAGTGCAGGGAGGACGCACGGAGAAATGCAGGCGACCACGTGGGACGGTGGCAAGAGGATCAGGAGAATGTGCGAAAGACGAGAGGGAGGATGAGGGCAGAGGCGGGGCCGGAGAAGGACAGCAGGAAGGAGAGAGGAGGAGAAAAGAATGACCGGAAGACGGGAACAGAGGGGACAGGAGGGGGAGGTTGGGGGAAAAAGAAGAGACAGAAATGGTTGACCGTTATAGCCCCAATGGGGGGTGCTACCCCGTCCGGGGCCTCCCACTTCCCCCCAATGTCAGGCTCAGGTTCCAAAAGATAATTTTATAAAAGAATACTGGCTTTTCCATTTCCATTCAAACAACAAAGGCGTGGGGGTCCCTGAGAAGGGTGGGAGGCACGGCCCCCTCCTCCTGCCCTGTCCAGTGTGAGTCTCTCGGTTTGAGGAGGAAGGGAGCACTAGGTCTGAGATCCCTGAGTGGGGCCCTTCCCCACGGTCCGACCACTCATTAGAGGAGGGGCCCCTGTGGCCGTAGGGGAAGAGGCCGTGTTCACAGCCGCCGGAGGTGGGAGAGGAAGAGGAGGAGA

7000

In [17]:
df = pd.DataFrame({'gene': ['ADAM10', 'ACHE'], 'locus': ['Chromosome 15', 'Chromosome 7'], 'disease': ["Alzheimer's Disease", "Alzheimer's Disease"], 'length': [160898, 7000]})
df

,gene,locus,disease,length
0,ADAM10,Chromosome 15,Alzheimer's Disease,160898
1,ACHE,Chromosome 7,Alzheimer's Disease,7000
